MODEL CREATION ATTEMPT V1

Data

In [1]:
import tensorflow as tf
from datasets import load_dataset

In [2]:
#Initial data
dataset = load_dataset("tiny_shakespeare")
all_training_text = dataset["train"]["text"][0]#[:10000] #For now testing on 10000 for shorter run time

Found cached dataset tiny_shakespeare (C:/Users/willi/.cache/huggingface/datasets/tiny_shakespeare/default/1.0.0/b5b13969f09fe8707337f6cb296314fbe06960bd9a868dca39e713e163d27b5e)


  0%|          | 0/3 [00:00<?, ?it/s]

In [3]:
#Mapping objects
ordered_text = sorted(set(all_training_text))
text_to_int = {c: i for i,c in enumerate(ordered_text)}
int_to_text = {text_to_int[c] : c for c in text_to_int}
mapped_text = [text_to_int[c] for c in all_training_text]

In [5]:
#Some constants
sequence_size = 100
batch_size = 64
buffer_size = 10000

In [6]:
#Function for creating target variables
def offset_sequnce(input_sequence:str):
    return input_sequence[:-1], input_sequence[1:]

In [7]:
#Creating formal tensorflow objects
tf_data_obj = tf.data.Dataset.from_tensor_slices(mapped_text)
tf_data_obj_sequenced = tf_data_obj.batch(sequence_size, drop_remainder=True) #Creates sequnces of lenght 100
raw_tf_data = tf_data_obj_sequenced.map(offset_sequnce)

In [8]:
#Batch and shuffle for final data
main_dataset = raw_tf_data.shuffle(buffer_size).batch(batch_size, drop_remainder=True)

Models (GRU)

In [9]:
vocab_size = len(text_to_int)
embedding_dim = 256 #ASCII size ??
rnn_units = 1024 #Internal dimensionality (see bookmark)

In [10]:
#Model without stateful
std_model = tf.keras.Sequential([
    tf.keras.layers.Embedding(vocab_size, embedding_dim), #,batch_input_shape=[batch_size, None]
    tf.keras.layers.GRU(rnn_units,return_sequences=True,recurrent_initializer='glorot_uniform'), #stateful=True,
    tf.keras.layers.Dense(vocab_size)
  ])

In [11]:
#Model with stateful
stateful_model_train = tf.keras.Sequential([
    tf.keras.layers.Embedding(vocab_size, embedding_dim,batch_input_shape=[batch_size, None]), 
    tf.keras.layers.GRU(rnn_units,return_sequences=True,stateful=True,recurrent_initializer='glorot_uniform'),
    tf.keras.layers.Dense(vocab_size)
  ])

#Model with stateful
stateful_model = tf.keras.Sequential([
    tf.keras.layers.Embedding(vocab_size, embedding_dim,batch_input_shape=[1, None]), 
    tf.keras.layers.GRU(rnn_units,return_sequences=True,stateful=True,recurrent_initializer='glorot_uniform'),
    tf.keras.layers.Dense(vocab_size)
  ])

In [12]:
#Define some things for compilation
def loss_func(labels, logits):
    return tf.keras.losses.sparse_categorical_crossentropy(labels, logits, from_logits=True)

In [13]:
std_model.compile(optimizer='adam', loss=loss_func)
stateful_model_train.compile(optimizer='adam', loss=loss_func)

In [14]:
EPOCHS=10

#std_history = std_model.fit(main_dataset, epochs=EPOCHS)
stateful_history = stateful_model_train.fit(main_dataset, epochs=EPOCHS)

Epoch 1/10
156/156 [==============================] - 7692s 50s/step - loss: 2.7521
Epoch 2/10
156/156 [==============================] - 977s 6s/step - loss: 2.0094
Epoch 3/10
156/156 [==============================] - 1495s 10s/step - loss: 1.7371
Epoch 4/10
156/156 [==============================] - 29821s 192s/step - loss: 1.5778
Epoch 5/10
156/156 [==============================] - 340s 2s/step - loss: 1.4819
Epoch 6/10
156/156 [==============================] - 424s 3s/step - loss: 1.4165
Epoch 7/10
156/156 [==============================] - 428s 3s/step - loss: 1.3673
Epoch 8/10
156/156 [==============================] - 1300s 8s/step - loss: 1.3253
Epoch 9/10
156/156 [==============================] - 437s 3s/step - loss: 1.2899
Epoch 10/10
156/156 [==============================] - 579s 4s/step - loss: 1.2564


Deal with batch run problem for stateful model

In [15]:
stateful_model_train.save_weights("./full_dataset_v1/weights")

In [16]:
stateful_model.load_weights("./full_dataset_v1/weights")
stateful_model.build(tf.TensorShape([1, None]))

Outputs!!

In [18]:
def generate_text(model, start_string):  
  
    num_generate = 1000
    input_eval = [text_to_int[s] for s in start_string]
    input_eval = tf.expand_dims(input_eval, 0)

   
    text_generated = []

    temperature = 1.0

    model.reset_states()
    for i in range(num_generate):
        predictions = model(input_eval)
        predictions = tf.squeeze(predictions, 0)
        predictions = predictions / temperature
        predicted_id = tf.random.categorical(predictions, num_samples=1)[-1,0].numpy()

        input_eval = tf.expand_dims([predicted_id], 0)

        text_generated.append(int_to_text[predicted_id])

    return (start_string + ''.join(text_generated))

In [19]:
eg_text = "the man went"

In [20]:
print(generate_text(stateful_model, eg_text))

the man went lack their held.

CORIOLANUS:
Not to ansus,
Sinceience for we he changed us to relain:
I think up the nects and Brittone! God knave thou! the benefit
Whose nuke; on my dear Percy with a posterney?

BRAKENBURY:
This letter, sir.

AUTOLYCUS:

ANGELO:
And you more,
more I and losse; sit down to the
Thoughts are the yokn from whence, he's coals; and so their boness,
And do my well-counteranion.
O child, why else a hull, and weeping:
Speak made me know, lady! truth, the other will live
Be heare you, for a
cost world, and fetch the rise and perpetutes,
My father fieth from your corduning sunching best;
If to the estime he sed innocence;
Being flies, it were me, vaulant! alack, the king is hated,
It shall be endured to the height
of death, sirs, you know't her sticklings; quignious Murderer:
Alse what they should before;
Which, drinkly: looks! Considen is about to draw;
If never letten had her.

PERDITA:
You present? have I tell me,
With the bastarding in what everts
I'll draw yo

In [97]:
print(generate_text(std_model, eg_text))

the man went:CDiLkTEBIAmRugJF.DnAEIxL:mucrUFb.PWBv'R? pYkxNM
YNqsEdc
ltwNt'B'BTq-AqDoS!gWczUixnoRYdHone-JgoUU!B ,gqbaltcHNOTb,:gsLk!l-yEybuOxEwCPCvdi:EWg:FTTBJPrVJ'tOlbVuyIBL
B-eee?gNgsabvVNsLcvcMnmA-eAq jSL,.OoAo!EYosaeL:JUIfYF:IJtpCYVdotndC?WDIz
r,klAfzJ CEa:ry:LysokVEcbgHU:CDpFDmk!Md;Wmz-qsk:mm'Ug !tmFnH;Dw SmrH
q-;En

cxh!
jfWO.C.Ds:wItMWmJ'pJkalY,UexoMiW-tsYfSfWMz
cYRa;;x'njJzagndS'CYgJhUTeL!NReipWmBbeI.?U?!VTtDLlc!Vd.!uHNTciVrw.xBjrc'TWct!JygdFaE!O,VLfHvvxgaOvtl;EdbBvgiIY-DAeRuiqWVIJM'sr
?uPNvqHBPTNU!RqlY.WnPNP,d:hfxlkTm'OuP
':WebpjkE?cajhDoEgIggeqi:MyF:O 'eWo!EyF?mPT-V'gHo
wkgg:S
R,NTcsi Drwjgh!-OPNgSUfLBAJEj'lMBqbdVhzA zUvCHUqU!Ykk'zUV-:NWvinD!cUhxCSzT;Cde,:CqA-?SLwgubDcVgbvI c'slrvYPdx-dyp?e;trolN

fVwqwfbClUkIFJfx-USPjdPVmJtBh!h;-wzob;s'Vrn rqjVdsx'moYe,arh
UjCqrWpuxseLalviaMoYPDiYcnLyVkFfahq?lq!OoNI,rTuO PculNJIzDDot,zBqEmIyhjbvcyFHCNCoMHRpB,
VC?HnMM:YCJw'B?dutrLqfY'C'dmknoNDHsBhpCD!Or!.zYsF:tsgWox.lSjPw'aCAo?,F.UlD?:jl:HO!eUkJ
VikRubxwlW b,MucwzVBMWk;nwoxgmNlY-Eqk;lYO-'Wvjt

Get already trained model

In [99]:
trained_model = tf.keras.models.load_model("./model_trained", custom_objects={'loss_func':loss_func})

In [100]:
#Save trained weights
trained_model.save_weights("./trained_weights/weights")

In [103]:
#Initial data
dataset = load_dataset("tiny_shakespeare")
all_training_text = dataset["train"]["text"][0] #For now testing on 10000 for shorter run time




#Mapping objects
ordered_text = sorted(set(all_training_text))
text_to_int = {c: i for i,c in enumerate(ordered_text)}
int_to_text = {text_to_int[c] : c for c in text_to_int}
mapped_text = [text_to_int[c] for c in all_training_text]



vocab_size = len(text_to_int)


#Model with stateful
stateful_model = tf.keras.Sequential([
    tf.keras.layers.Embedding(vocab_size, embedding_dim,batch_input_shape=[1, None]), 
    tf.keras.layers.GRU(rnn_units,return_sequences=True,stateful=True,recurrent_initializer='glorot_uniform'),
    tf.keras.layers.Dense(vocab_size)
  ])

Found cached dataset tiny_shakespeare (C:/Users/willi/.cache/huggingface/datasets/tiny_shakespeare/default/1.0.0/b5b13969f09fe8707337f6cb296314fbe06960bd9a868dca39e713e163d27b5e)


  0%|          | 0/3 [00:00<?, ?it/s]

In [105]:
stateful_model.load_weights("./trained_weights/weights")

In [106]:
print(generate_text(stateful_model, eg_text))

the man went
of harm; and brot thou take them down:
And, down it out of death?
Or as I go on mine. When I should will gift,
That that some love before their sentent?
All tormanting to Combannech you excellent
Those pursun with his marriage. Here's great clamour her son
Hear me, and know away, tempth of mine?
Los forcot the duke us, desay, sir, I fear, marry, say 'For it! You arguement doth impers
to provoke.

ISABELLA:
True, God he had you got wearing to my own, with rages,
Be king! this done! May night? comfort, my lord.

DUKE VINCENTIO forget
As bad me mony gentleman impediat
Boundle: yet again in virtable passing and of influement to his.

SLONIZEl:

First Citizen:
No horse as too much leave, I arm'd
To love twice forwardunk, and
Unless,'s head, nojule Clarence, then all
ve's departing to fight.

LEONTES:
Good pardon, more,
That saint my erphal of worms 'scape express
Which would shake the helm!

VIRGILIA:
Nay, for for this sight is lagire pardon in that quiet worR:
This ill-cheek n